In [1]:
# RESETEAR TODO PARA PARTIR LIMPIO EN COLAB

import os, shutil

# 1) Ir a /content
%cd /content

# 2) Borrar cualquier clone previo duplicado
if os.path.exists("proyecto_aplicado_preservantes"):
    shutil.rmtree("proyecto_aplicado_preservantes")
    print("🗑️ Carpeta borrada: proyecto_aplicado_preservantes")

# 3) Clonar de nuevo desde tu GitHub
!git clone https://github.com/nalpata/proyecto_aplicado_preservantes.git

# 4) Entrar a la carpeta correcta
%cd proyecto_aplicado_preservantes

print("\n🎉 Listo. Ahora estamos en el repo correcto sin duplicados.")
!ls


/content
Cloning into 'proyecto_aplicado_preservantes'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (115/115), done.
remote: Compressing objects: 100% (104/104), done.
remote: Total 115 (delta 34), reused 77 (delta 10), pack-reused 0 (from 0)
Receiving objects: 100% (115/115), 19.32 MiB | 44.76 MiB/s, done.
Resolving deltas: 100% (34/34), done.
/content/proyecto_aplicado_preservantes

🎉 Listo. Ahora estamos en el repo correcto sin duplicados.
 02_chunking.ipynb	    MEJORAS_CHUNKING.md    RESUMEN_PROBLEMAS.md
 CHECKLIST_INSTALACION.md   notebooks		   run_pipeline.py
 data			   'Pauta proyecto.pdf'    SOLUCION_INSTALACION.md
 DIAGRAMAS_HITO_1.md	    PROBLEMAS_COMUNES.md   src
 examples.py		    QUICK_START.md	   streamlit_app.py
 FIX_CHROMADB.md	    README.md		   test_improvements.py
 install_dependencies.sh    requirements.txt	   TESTING_LOCAL.md
 install_fix.sh		    RESUMEN_FINAL.md	   test_single_pdf.py
 INSTRUCCIONES_PRUEBA.md    RESUMEN_HITO_1.md


In [17]:
import os

print("ANTES, carpeta actual:", os.getcwd())
%cd /content/proyecto_aplicado_preservantes
print("AHORA, carpeta actual:", os.getcwd())


ANTES, carpeta actual: /content
/content/proyecto_aplicado_preservantes
AHORA, carpeta actual: /content/proyecto_aplicado_preservantes


# Lesson 2: Document Chunking Strategies

**Objective**: Learn how to effectively segment documents for better retrieval performance.

**Topics**:

- Chunking techniques: token-level, sentence-level, semantic-level
- Balancing context preservation with retrieval precision
- Small2Big and sliding window techniques

**Practical Task**: Implement chunking strategies on a sample dataset.

**Resources**:
- The five levels of chunking
- A guide to chunking
- [Visualizing chunking](https://chunkviz.up.railway.app/)
- [Full stack retrieval](https://github.com/FullStackRetrieval-com/RetrievalTutorials/blob/a4570f3c4883eb9b835b0ee18990e62298f518ef/tutorials/LevelsOfTextSplitting/5_Levels_Of_Text_Splitting.ipynb)

## Data loading, Chunking techniques and Text processing


### Level 1 - Character splitting

This is the simplest and most basic *chunking* method, it is the process by which we only divide the text into *chunks* of size *N* characters regardless of their content or form.

This method is **NOT** recommended for any application, but it can be a good starting point to understand the fundamentals.

- Pros: easy and fast
- Cons: Very rigid and does not take into account the structure of the text

Concepts:

- **Chunk size**: The number of characters that defines the length of the chunk.
- **Chunk overlap**: The amount of characters that must overlap in sequential chunks. This is to avoid cutting the context into multiple parts, losing the semantic meaning of some passage. This will create duplicates between chunks.

#### Manual example

In [2]:
from pprint import pprint

In [3]:
text = "This is the text I would like to chunk up. It is the example text for this exercise"

# Create an empty list to store the chunks
chunks = []

# Define the chunk size
chunk_size = 35

# Iterate through the text in chunk_size ranges
for i in range(0, len(text), chunk_size):
    chunk = text[i:i + chunk_size]
    chunks.append(chunk)
chunks

['This is the text I would like to ch',
 'unk up. It is the example text for ',
 'this exercise']

In [4]:
len(chunks[0])

35

#### Using the LangChain framework

- Normally for RAG applications we don't work with strings, we work with **Document** type objects. These objects contain the text we want but also contain additional metadata that allows for more complex filtering.

In [8]:
!pip install langchain-text-splitters


In [9]:
from langchain_text_splitters import CharacterTextSplitter


We load the LangChain text splitter, we need to specify the chunk overlap and separator, if we don't provide them we will have unwanted results.

In [10]:
# Sin overlap
text_splitter = CharacterTextSplitter(
                    chunk_size = 35,
                    chunk_overlap=0,
                    separator='',
                    strip_whitespace=True)

In [11]:
text_splitter.create_documents([text])

[Document(metadata={}, page_content='This is the text I would like to ch'),
 Document(metadata={}, page_content='unk up. It is the example text for'),
 Document(metadata={}, page_content='this exercise')]

![image.png](attachment:image.png)

If we observe, we achieved the same result as with the manual option, but now we have data structures that are compatible with the rest of the framework. Additionally, we were able to do some string cleaning, in the case of the whitespace on the second line, using strip_whitespace=True.

In [12]:
# Con overlap
text_splitter = CharacterTextSplitter(
                    chunk_size = 35,
                    chunk_overlap=4,
                    separator='',
                    strip_whitespace=True)
text_splitter.create_documents([text])

[Document(metadata={}, page_content='This is the text I would like to ch'),
 Document(metadata={}, page_content='o chunk up. It is the example text'),
 Document(metadata={}, page_content='ext for this exercise')]

![image.png](attachment:image.png)

If we look closely, we have the same chunks, but between chunk 1 and 2 and chunk 2 and 3 some characters are repeated.

### Level 2: Recursive Character Text Splitting

One of the cons of the previous method is that we don't take into account the structure of the documents. We simply count characters and cut.

**Recursive Character Text Splitter** helps us consider at least the structure of the text, we will specify a series of separators that will be used to separate our document.

These are the available separators:

- "\n\n" - Double line, a common paragraph separator.
- "\n" - Line break, a common sentence separator.
- " " - Whitespace, a common word separator.
- "" - Empty character, a common character separator.

This is the most effective solution to start prototyping an application quickly. If we don't know which one to use, this is the first option to try.

In [13]:
# Imports
from langchain_text_splitters import RecursiveCharacterTextSplitter

Let's test with a longer text

In [14]:
text = """
One of the most important things I didn't understand about the world when I was a child is the degree to which the returns for performance are superlinear.

Teachers and coaches implicitly told us the returns were linear. "You get out," I heard a thousand times, "what you put in." They meant well, but this is rarely true. If your product is only half as good as your competitor's, you don't get half as many customers. You get no customers, and you go out of business.

It's obviously true that the returns for performance are superlinear in business. Some think this is a flaw of capitalism, and that if we changed the rules it would stop being true. But superlinear returns for performance are a feature of the world, not an artifact of rules we've invented. We see the same pattern in fame, power, military victories, knowledge, and even benefit to humanity. In all of these, the rich get richer. [1]
"""

In [15]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 65, chunk_overlap=0)

In [ ]:
split_recursive_character = text_splitter.create_documents([text])
split_recursive_character

[Document(metadata={}, page_content="One of the most important things I didn't understand about the"),
 Document(metadata={}, page_content='world when I was a child is the degree to which the returns for'),
 Document(metadata={}, page_content='performance are superlinear.'),
 Document(metadata={}, page_content='Teachers and coaches implicitly told us the returns were linear.'),
 Document(metadata={}, page_content='"You get out," I heard a thousand times, "what you put in." They'),
 Document(metadata={}, page_content='meant well, but this is rarely true. If your product is only'),
 Document(metadata={}, page_content="half as good as your competitor's, you don't get half as many"),
 Document(metadata={}, page_content='customers. You get no customers, and you go out of business.'),
 Document(metadata={}, page_content="It's obviously true that the returns for performance are"),
 Document(metadata={}, page_content='superlinear in business. Some think this is a flaw of'),
 Document(metadata=

If we look closely, several of the chunks end with a **"."**. This is because they are probably the end of a paragraph and the splitter first checks if there is a double line break.

Once the paragraphs are separated, the splitter looks at the chunk size, if it's too big, it separates it by the next separator. If the chunk is still too long, it continues with all the separators.

In [16]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 450, chunk_overlap=0)
text_splitter.create_documents([text])

[Document(metadata={}, page_content="One of the most important things I didn't understand about the world when I was a child is the degree to which the returns for performance are superlinear."),
 Document(metadata={}, page_content='Teachers and coaches implicitly told us the returns were linear. "You get out," I heard a thousand times, "what you put in." They meant well, but this is rarely true. If your product is only half as good as your competitor\'s, you don\'t get half as many customers. You get no customers, and you go out of business.'),
 Document(metadata={}, page_content="It's obviously true that the returns for performance are superlinear in business. Some think this is a flaw of capitalism, and that if we changed the rules it would stop being true. But superlinear returns for performance are a feature of the world, not an artifact of rules we've invented. We see the same pattern in fame, power, military victories, knowledge, and even benefit to humanity. In all of these, th

![image.png](attachment:image.png)



## Level 3: Document Specific Splitting

We continue increasing the complexity, now let's look at different types of documents other than .txt. What happens if we have images? Is it a PDF? Or code?

The previous approaches won't work very well with this type of documents.

This level is all about adjusting the **chunking** strategy to our use case. For now we will only see how we handle complex PDFs. But we can also process other types of files, such as Markdown, Python, JS, etc.

### Markdown

In [17]:
!pip install -U langchain langchain-community langchain-text-splitters


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.8/473.8 kB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.7 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.0.7
    Uninstalling langchain-core-1.0.7:
      Successfully uninstalled langchain-core-1.0.7
  Attempting uninstall: langchain
    Found existing installation: langchain 1.0.8
    Uninstalling langchain-1.0.8:
      Successfully uninstalled langchain-1.0.8
ERROR: pip's depen

In [1]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader

In [2]:
!pip install "unstructured[md]"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 19.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.8/167.8 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 92.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.9/207.9 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.5/329.5 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 11.7 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=d014f9172750ab9d2c1d491ce8a6a9f5a62df48f14a10b46ab46ddbdd841b9fc
  Stored in directory: /root/.cache/pip/wheels/c1/67/88/e844b5b022812e15a52e4eaa38a1e709e99f06f6639d7e3ba7
Successfully built langdetect


In [3]:
import os

# Crear carpeta data si no existe
os.makedirs("data", exist_ok=True)

# Crear un archivo markdown de ejemplo
contenido = """# Título de ejemplo

Este es un pequeño documento en **Markdown**.

- Punto 1
- Punto 2
- Punto 3

## Subtítulo

Más texto de ejemplo para probar el chunking.
"""

with open("data/example.md", "w", encoding="utf-8") as f:
    f.write(contenido)

print("Archivo data/example.md creado ✅")


Archivo data/example.md creado ✅


In [4]:
markdown_path = "data/example.md"
loader = UnstructuredMarkdownLoader(markdown_path, mode="elements")
data = loader.load()
data

[Document(metadata={'source': 'data/example.md', 'category_depth': 0, 'languages': ['spa'], 'file_directory': 'data', 'filename': 'example.md', 'filetype': 'text/markdown', 'last_modified': '2025-11-26T02:46:19', 'category': 'Title', 'element_id': '66b7f8c1c2934ce7e1bd492787be2650'}, page_content='Título de ejemplo'),
 Document(metadata={'source': 'data/example.md', 'emphasized_text_contents': ['Markdown'], 'emphasized_text_tags': ['b'], 'languages': ['spa'], 'file_directory': 'data', 'filename': 'example.md', 'filetype': 'text/markdown', 'last_modified': '2025-11-26T02:46:19', 'parent_id': '66b7f8c1c2934ce7e1bd492787be2650', 'category': 'UncategorizedText', 'element_id': '7c9bcd284768d4b68030a3c03fb7b230'}, page_content='Este es un pequeño documento en Markdown.'),
 Document(metadata={'source': 'data/example.md', 'category_depth': 1, 'languages': ['spa'], 'file_directory': 'data', 'filename': 'example.md', 'filetype': 'text/markdown', 'last_modified': '2025-11-26T02:46:19', 'parent_id

In [5]:
pprint(data[6].page_content)

Pretty printing has been turned OFF


In [6]:
pprint(data[6].metadata)


Pretty printing has been turned ON


### Python

In [7]:
from langchain_text_splitters import PythonCodeTextSplitter

In [8]:
python_text = """
class Person:
  def __init__(self, name, age):
    self.name = name
    self.age = age

p1 = Person("John", 36)

for i in range(10):
    print (i)
"""

In [9]:
python_splitter = PythonCodeTextSplitter(chunk_size=100, chunk_overlap=0)

In [10]:
pprint(python_splitter.create_documents([python_text]))


Pretty printing has been turned OFF


## Level 4: Semantic Chunking

Considering that the previous methodologies only consider dividing the text into fixed lengths, or defined by static delimiters. Doesn't it seem strange that none of those approaches take into account the content of the documents?

There is a better way.

Embeddings represent the semantic meaning of a string. By themselves, they are not very useful, but when compared with others, it is possible to infer the relationship between different strings.

The hypothesis then is that semantically similar chunks should be grouped together.

In [18]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import glob

# Buscar todos los PDFs de preservantes
file_paths = glob.glob("data/pdfs/*.pdf")

# Cargar documentos
docs = []
for path in file_paths:
    loader = PyPDFLoader(path)
    docs.extend(loader.load())

# Chunking semántico básico (por tamaño + overlap)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(docs)

len(chunks), chunks[:2]  # mostramos solo los 2 primeros


(2145, [Document(metadata={'producer': 'Adobe PDF Library 10.0.1', 'creator': 'Adobe InDesign CS6 (Windows)', 'creationdate': '2015-03-01T16:45:26+05:30', 'moddate': '2015-03-01T16:45:27+05:30', 'subject': '.', 'title': '.', 'trapped': '/False', 'source': 'data/pdfs/cf6896f938cab26391a2541c2ecada647fd2.pdf', 'total_pages': 8, 'page': 0, 'page_label': '366'}, page_content='366  Emir. J. Food Agric ● Vol 28 ● Issue 6 ● 2016\nFood preservatives – An overview on applications and \nside effects\nMaria Manuela Silva1*, Fernando Cebola Lidon2\n1ESE Almeida Garrett, Grupo Universidade Lusófona, COFAC, Rua de São Paulo, Lisboa, Portugal, 2GeoBioTec, Departamento de Ciências \nda Terra, Faculdade de Ciências e Tecnologia, Universidade Nova de Lisboa, Caparica, Portugal\nINTRODUCTION\nFood additives are substances added intentionally to \nfoodstuff, to increase the durability of  the product and \nenhance or modify its properties, including its appearance, \nflavor or structure, provided it does 

Para explotar la información contenida en los documentos de preservantes, realizamos un proceso de chunking semántico, que divide el PDF en fragmentos de 1000 caracteres con solape de 200. Esto permite preservar el contexto entre secciones relacionadas, mejorando el rendimiento posterior de los embeddings y del RAG.

In [19]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import glob

file_paths = glob.glob("data/pdfs/*.pdf")
print("PDFs encontrados:", file_paths)

docs = []
for path in file_paths:
    loader = PyPDFLoader(path)
    docs.extend(loader.load())

if docs:
    print("Cantidad de páginas cargadas:", len(docs))
    docs[0]
else:
    print(" No se cargó ningún documento. Revisa la ruta de los PDFs.")


PDFs encontrados: ['data/pdfs/cf6896f938cab26391a2541c2ecada647fd2.pdf', 'data/pdfs/j.ijfoodmicro.2013.06.025.pdf', 'data/pdfs/Effects of Acidification and Preservatives on Microbial Growth Puree.pdf', 'data/pdfs/Food aditives1.pdf', 'data/pdfs/fmicb-13-930400.pdf', 'data/pdfs/benzoate-sorbate.pdf', 'data/pdfs/1-s2.0-S2212429225020413-main.pdf', 'data/pdfs/FTB-61-212.pdf', 'data/pdfs/Prop Ca y s. potasio.pdf', 'data/pdfs/s13568-018-0536-0.pdf', 'data/pdfs/Preservantes.pdf', 'data/pdfs/processes-11-00590.pdf', 'data/pdfs/j.ijfoodmicro.2020.108952.pdf', 'data/pdfs/Molecules to food, preservatives.pdf', 'data/pdfs/antibiotics-08-00208.pdf', 'data/pdfs/The Scientific World Journal - 2022 - Teshome - Potentials of Natural Preservatives to Enhance Food Safety and Shelf Life .pdf', 'data/pdfs/1-s2.0-S2405844023042287-main.pdf', 'data/pdfs/AntimicrobialActivityofSpiceextracts.pdf', 'data/pdfs/IJFS2018-8410747.pdf']
Cantidad de páginas cargadas: 475


In [20]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

basic_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)

basic_chunks = basic_splitter.split_documents(docs)

print("Chunks básicos:", len(basic_chunks))
basic_chunks[0]


Chunks básicos: 2145


Document(metadata={'producer': 'Adobe PDF Library 10.0.1', 'creator': 'Adobe InDesign CS6 (Windows)', 'creationdate': '2015-03-01T16:45:26+05:30', 'moddate': '2015-03-01T16:45:27+05:30', 'subject': '.', 'title': '.', 'trapped': '/False', 'source': 'data/pdfs/cf6896f938cab26391a2541c2ecada647fd2.pdf', 'total_pages': 8, 'page': 0, 'page_label': '366'}, page_content='366  Emir. J. Food Agric ● Vol 28 ● Issue 6 ● 2016\nFood preservatives – An overview on applications and \nside effects\nMaria Manuela Silva1*, Fernando Cebola Lidon2\n1ESE Almeida Garrett, Grupo Universidade Lusófona, COFAC, Rua de São Paulo, Lisboa, Portugal, 2GeoBioTec, Departamento de Ciências \nda Terra, Faculdade de Ciências e Tecnologia, Universidade Nova de Lisboa, Caparica, Portugal\nINTRODUCTION\nFood additives are substances added intentionally to \nfoodstuff, to increase the durability of  the product and \nenhance or modify its properties, including its appearance, \nflavor or structure, provided it does not detr

In [21]:
##Chunking “estructural” usando títulos / secciones

import re
from langchain_core.documents import Document

# Patrón simple para detectar líneas tipo título (ajústalo según tus PDFs)
pattern_titulo = r"\n([A-ZÁÉÍÓÚÑ][A-ZÁÉÍÓÚÑ0-9 \-]{5,})\n"

def split_by_titles(docs, regex_pattern):
    chunks = []
    for doc in docs:
        text = doc.page_content

        # Separamos por títulos con re.split (manteniendo los títulos)
        parts = re.split(regex_pattern, text)

        # Reconstruimos: título + cuerpo
        # parts = ["textoInicial", "TITULO", "texto1", "TITULO", "texto2", ...]
        for i in range(1, len(parts), 2):
            title = parts[i].strip()
            body = parts[i+1].strip() if i+1 < len(parts) else ""
            if body:
                chunks.append(Document(
                    page_content=f"{title}\n{body}",
                    metadata=doc.metadata
                ))
    return chunks

# Ejecutamos nuestro split por títulos
struct_chunks = split_by_titles(docs, pattern_titulo)

print("Chunks estructurales (por títulos):", len(struct_chunks))
struct_chunks[0]


Chunks estructurales (por títulos): 132


Document(metadata={'producer': 'Adobe PDF Library 10.0.1', 'creator': 'Adobe InDesign CS6 (Windows)', 'creationdate': '2015-03-01T16:45:26+05:30', 'moddate': '2015-03-01T16:45:27+05:30', 'subject': '.', 'title': '.', 'trapped': '/False', 'source': 'data/pdfs/cf6896f938cab26391a2541c2ecada647fd2.pdf', 'total_pages': 8, 'page': 0, 'page_label': '366'}, page_content='INTRODUCTION\nFood additives are substances added intentionally to \nfoodstuff, to increase the durability of  the product and \nenhance or modify its properties, including its appearance, \nflavor or structure, provided it does not detract from its \nnutritional value. Substances can be of  natural or synthetic \norigin, usually without appreciable nutritional value, that \nare added to food in small amounts during the manufacture \n(industrial change or during packaging).\nThe use of  food additives is regulated by specific laws \n(namely in the European Union - EU), considering the \nfood where it can be applied, maximum u

In [22]:
!pip install -U langchain-experimental sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.6/209.6 kB 5.7 MB/s eta 0:00:00


In [23]:
##Definir embeddings y SemanticChunker

from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.embeddings import HuggingFaceEmbeddings

# Modelo de embeddings gratuito y liviano
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

semantic_splitter = SemanticChunker(
    embeddings=embedding_model,
    breakpoint_threshold_type="percentile",  # dónde "cortar" según cambios semánticos
    breakpoint_threshold_amount=95,          # 95 = pocos cortes, chunks más largos
)

semantic_chunks = semantic_splitter.split_documents(docs)

print("Chunks semánticos:", len(semantic_chunks))
semantic_chunks[0]


/tmp/ipython-input-4064929628.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Chunks semánticos: 1650


Document(metadata={'producer': 'Adobe PDF Library 10.0.1', 'creator': 'Adobe InDesign CS6 (Windows)', 'creationdate': '2015-03-01T16:45:26+05:30', 'moddate': '2015-03-01T16:45:27+05:30', 'subject': '.', 'title': '.', 'trapped': '/False', 'source': 'data/pdfs/cf6896f938cab26391a2541c2ecada647fd2.pdf', 'total_pages': 8, 'page': 0, 'page_label': '366'}, page_content='366  Emir.')

In [24]:
for pct in [80, 90, 95]:
    splitter = SemanticChunker(
        embeddings=embedding_model,
        breakpoint_threshold_type="percentile",
        breakpoint_threshold_amount=pct,
    )
    ch = splitter.split_documents(docs)
    print(f"Percentile {pct}: {len(ch)} chunks")


Percentile 80: 4421 chunks
Percentile 90: 2551 chunks
Percentile 95: 1650 chunks


In [28]:
!pip install langchain-text-splitters


In [31]:
import re
from langchain_core.documents import Document

def regex_split_documents(docs, pattern):
    new_docs = []

    for doc in docs:
        # Dividir usando regex (manteniendo el separador)
        parts = re.split(pattern, doc.page_content)

        # Volver a formar los chunks (sección + texto)
        for i in range(1, len(parts), 2):
            section = parts[i].strip()
            content = parts[i+1].strip()

            if content:
                new_docs.append(Document(
                    page_content=f"{section}\n{content}",
                    metadata=doc.metadata
                ))
    return new_docs


In [32]:
pattern = r"(?i)(abstract|introduction|methods?|methodology|results?|discussion|conclusions?)"
struct_chunks = regex_split_documents(docs, pattern)

print("Secciones encontradas:", len(struct_chunks))
struct_chunks[0]


Secciones encontradas: 567


Document(metadata={'producer': 'Adobe PDF Library 10.0.1', 'creator': 'Adobe InDesign CS6 (Windows)', 'creationdate': '2015-03-01T16:45:26+05:30', 'moddate': '2015-03-01T16:45:27+05:30', 'subject': '.', 'title': '.', 'trapped': '/False', 'source': 'data/pdfs/cf6896f938cab26391a2541c2ecada647fd2.pdf', 'total_pages': 8, 'page': 0, 'page_label': '366'}, page_content='INTRODUCTION\nFood additives are substances added intentionally to \nfoodstuff, to increase the durability of  the product and \nenhance or modify its properties, including its appearance, \nflavor or structure, provided it does not detract from its \nnutritional value. Substances can be of  natural or synthetic \norigin, usually without appreciable nutritional value, that \nare added to food in small amounts during the manufacture \n(industrial change or during packaging).\nThe use of  food additives is regulated by specific laws \n(namely in the European Union - EU), considering the \nfood where it can be applied, maximum u

In [35]:
##Chunking jerárquico (estructura + tamaño)

from langchain_text_splitters import RecursiveCharacterTextSplitter

fine_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=150
)

hier_chunks = fine_splitter.split_documents(struct_chunks)
print("Chunks finales:", len(hier_chunks))
hier_chunks[0]

Chunks finales: 1339


Document(metadata={'producer': 'Adobe PDF Library 10.0.1', 'creator': 'Adobe InDesign CS6 (Windows)', 'creationdate': '2015-03-01T16:45:26+05:30', 'moddate': '2015-03-01T16:45:27+05:30', 'subject': '.', 'title': '.', 'trapped': '/False', 'source': 'data/pdfs/cf6896f938cab26391a2541c2ecada647fd2.pdf', 'total_pages': 8, 'page': 0, 'page_label': '366'}, page_content='INTRODUCTION\nFood additives are substances added intentionally to \nfoodstuff, to increase the durability of  the product and \nenhance or modify its properties, including its appearance, \nflavor or structure, provided it does not detract from its \nnutritional value. Substances can be of  natural or synthetic \norigin, usually without appreciable nutritional value, that \nare added to food in small amounts during the manufacture \n(industrial change or during packaging).\nThe use of  food additives is regulated by specific laws \n(namely in the European Union - EU), considering the \nfood where it can be applied, maximum u

In [36]:
##Comparaciòn de  las tres estrategias
print("Chunks básicos (solo tamaño):    ", len(basic_chunks))
print("Chunks estructurales (títulos):  ", len(struct_chunks))
print("Chunks semánticos (embeddings):  ", len(semantic_chunks))
print("Chunks jerárquicos (2 pasos):    ", len(hier_chunks))


Chunks básicos (solo tamaño):     2145
Chunks estructurales (títulos):   567
Chunks semánticos (embeddings):   1650
Chunks jerárquicos (2 pasos):     1339


**CONCLUSION:**

La estrategia jerárquica (estructura + tamaño) produce chunks óptimos en número y calidad:
respeta la lógica científica de los artículos y evita chunks demasiado grandes o incoherentes, mejorando notablemente la precisión en embeddings y en el futuro modelo RAG.

Esto la hace ideal para el análisis de preservantes alimentarios, porque:

Los papers científicos tienen secciones bien definidas

Cada sección contiene información coherente y especializada

El chunking jerárquico permite:

Mejor recuperación de evidencias

Mejor respuesta explicativa del modelo

Posible citación automática por sección en el RAG

## Level 5: Agentic Chunking

Taking level 4 a bit further, can we give the same instruction to an LLM to do it as a human would?

But, how would a human do chunking? Based on level 4 we could follow these steps:

1. We take a sheet of paper
2. We would start with the first sentence and assume it will be the first chunk, since we don't have any other.
3. We continue reading the document and evaluating if it's a new chunk or part of the previous chunk or an existing one. If it's similar to an existing one, we concatenate it to that chunk, if not, we create a new one.
4. We continue doing this until we finish the document.

In [38]:
text_document = docs[0].page_content


In [41]:
from langchain_community.document_loaders import PyPDFLoader

pdf_path = "data/pdfs/FTB-61-212.pdf"
loader = PyPDFLoader(pdf_path)
docs_pdf = loader.load()               # lista de Document, uno por página

len(docs_pdf), docs_pdf[0].page_content[:500]


(14, 'April-June 2023 | Vol. 61 | No. 2\n212\nEssential Oil and Plant Extracts as Preservatives and Natural \nAntioxidants Applied to Meat and Meat Products: A Review \nreview \nISSN 1330-9862\nhttps://doi.org/10.17113/ftb.61.02.23.7883\nGabriela Aguiar  \nCampolina1\n , Maria das \nGraças Cardoso2*\n , Alex \nRodrigues Silva Caetano2\n ,  \nDavid Lee Nelson3\n  and \nEduardo Mendes Ramos1\n1 Food Sciences Department, Federal \nUniversity of Lavras (UFLA), Professor \nEdmir Sá Santos Rotatory Clover, \n3037 Lavras, MG, B')

In [42]:
text_document = "\n\n".join(d.page_content for d in docs_pdf)
len(text_document)


67421

In [54]:
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass("🔑 Enter your OpenAI API key: ")

KeyboardInterrupt: Interrupted by user

In [49]:
!pip install langchain-openai

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)


In [50]:
class AgenticChunker:
    def __init__(self, llm):
        self.llm = llm

    def generate_summary(self, text: str) -> str:
        """Pide al LLM un mini-resumen del chunk."""
        prompt = f"Summarize this scientific text in one sentence:\n\n{text}"
        result = self.llm.invoke(prompt)
        # En LangChain moderno, el resultado tiene .content
        return getattr(result, "content", str(result))

    def chunk_document(self, text: str, target_chars: int = 2000, overlap_chars: int = 200):
        """
        Divide el texto en chunks solapados de tamaño fijo y genera un resumen para cada uno.
        Esto imita un 'agentic chunking' muy simple: el modelo entiende cada bloque y lo resume.
        """
        # 1) Dividimos por párrafos (simulando la lectura humana)
        paragraphs = text.split("\n\n")
        full_text = " ".join(p.strip() for p in paragraphs if p.strip())

        chunks = []
        start = 0

        # 2) Vamos avanzando como si leyéramos con una regla de caracteres
        while start < len(full_text):
            end = start + target_chars
            chunk_text = full_text[start:end]
            chunks.append(chunk_text)
            # 3) Solapamos un poco para no cortar ideas de golpe
            start = max(0, start + target_chars - overlap_chars)

        # 4) Para cada chunk pedimos un resumen al LLM
        chunk_objects = []
        for i, ch in enumerate(chunks):
            summary = self.generate_summary(ch)
            chunk_objects.append({
                "id": i,
                "text": ch,
                "summary": summary
            })
        return chunk_objects


In [52]:
agentic_chunker = AgenticChunker(llm)

agentic_chunks = agentic_chunker.chunk_document(
    text_document,
    target_chars=2000,   # tamaño aproximado de cada chunk
    overlap_chars=200    # solapamiento
)

len(agentic_chunks)


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

# Text processing



### Tokenization, POS and Dependency Parsing

In [53]:
$ pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.8.0/en_core_web_sm-3.8.0.tar.gz

SyntaxError: invalid syntax (ipython-input-2763435240.py, line 1)

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

file_path = (
    "data/Regulaciones cacao y chocolate 2003.pdf"
)
loader = PyPDFLoader(file_path)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [ ]:
pprint(splits[0].page_content)

('Status:  This is the original version (as it was originally made).\n'
 'STATUTORY INSTRUMENTS\n'
 '2003 No. 1659\n'
 'FOOD, ENGLAND\n'
 'The Cocoa and Chocolate Products (England) Regulations 2003\n'
 'Made       -      -      -      - 25th June 2003\n'
 'Laid before Parliament 3rd July 2003\n'
 'Coming into force       -      - 3rd August 2003\n'
 'The Secretary of State, in exercise of the powers conferred by sections '
 '16(1)(e), 17(1), 26(1) and (3)\n'
 'and 48(1) of the Food Safety Act 1990(1) and now vested in him(2) and of all '
 'other powers enabling\n'
 'him in that behalf, having had regard in accordance with section 48(4A) of '
 'that Act to relevant\n'
 'advice given by the Food Standards Agency, and after consultation both as '
 'required by Article 9\n'
 'of Regulation (EC) No. 178/2002 of the European Parliament and of the '
 'Council laying down the\n'
 'general principles and requirements of food law, establishing the European '
 'Food Safety Authority\n'
 'and lay

In [ ]:
import spacy
import en_core_web_sm

nlp = en_core_web_sm.load()
single_doc = nlp(splits[0].page_content)

In [ ]:
def tokenize_text(doc):
    return[
        {
            "text": token.text,
            "position": token.pos_, #part of speech
            "dep": token.dep_, #dependency relation
        }
        for token in doc
    ]

## Token Analysis Function

The `tokenize_text` function extracts linguistic information from a spaCy-processed document:

**Purpose**: Convert spaCy tokens into structured data for text analysis and processing.

**Input**: A spaCy `Doc` object (processed text document)

**Output**: List of dictionaries, where each dictionary contains:

- **`"text"`**: The actual word or punctuation mark (e.g., "The", "cat", ".")
- **`"position"`**: Part-of-speech tag indicating grammatical role:
  - `"DET"` = Determiner (the, a, an)
  - `"NOUN"` = Noun (cat, house, idea)
  - `"VERB"` = Verb (run, think, is)
  - `"ADJ"` = Adjective (quick, beautiful)
  - `"PUNCT"` = Punctuation (., !, ?)
- **`"dep"`**: Dependency relationship showing how words connect:
  - `"det"` = Determiner relationship
  - `"nsubj"` = Nominal subject
  - `"amod"` = Adjectival modifier
  - `"punct"` = Punctuation

**Example Output**:
```python
[
    {'text': 'The', 'position': 'DET', 'dep': 'det'},
    {'text': 'quick', 'position': 'ADJ', 'dep': 'amod'},
    {'text': 'fox', 'position': 'NOUN', 'dep': 'nsubj'},
    {'text': '.', 'position': 'PUNCT', 'dep': 'punct'}
]
```

This structured representation enables intelligent text processing for chunking strategies that preserve linguistic meaning.

In [ ]:
for token in tokenize_text(single_doc):
    print(token)

{'text': 'Status', 'position': 'NOUN', 'dep': 'dep'}
{'text': ':', 'position': 'PUNCT', 'dep': 'punct'}
{'text': ' ', 'position': 'SPACE', 'dep': 'dep'}
{'text': 'This', 'position': 'PRON', 'dep': 'nsubj'}
{'text': 'is', 'position': 'AUX', 'dep': 'ROOT'}
{'text': 'the', 'position': 'DET', 'dep': 'det'}
{'text': 'original', 'position': 'ADJ', 'dep': 'amod'}
{'text': 'version', 'position': 'NOUN', 'dep': 'attr'}
{'text': '(', 'position': 'PUNCT', 'dep': 'punct'}
{'text': 'as', 'position': 'SCONJ', 'dep': 'mark'}
{'text': 'it', 'position': 'PRON', 'dep': 'nsubjpass'}
{'text': 'was', 'position': 'AUX', 'dep': 'auxpass'}
{'text': 'originally', 'position': 'ADV', 'dep': 'advmod'}
{'text': 'made', 'position': 'VERB', 'dep': 'advcl'}
{'text': ')', 'position': 'PUNCT', 'dep': 'punct'}
{'text': '.', 'position': 'PUNCT', 'dep': 'punct'}
{'text': '\n', 'position': 'SPACE', 'dep': 'dep'}
{'text': 'STATUTORY', 'position': 'PROPN', 'dep': 'compound'}
{'text': 'INSTRUMENTS', 'position': 'NOUN', 'dep':

### Stopwords removal


In [ ]:
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS

def remove_stopwords(doc):
    return [token for token in doc if token.text.lower() not in spacy_stopwords]

filtered_tokens = remove_stopwords(single_doc)

In [ ]:
# Print the first few filtered tokens to verify
for token in filtered_tokens[:10]:
    print(token.text)

Status
:
 
original
version
(
originally
)
.




### Stemming

In [ ]:
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()

#example
text = "running runners run easily jumper jumps jumping"

doc = nlp(text)

# apply stemming
stemmed_tokens = [stemmer.stem(token.text) for token in doc]

stem_text = " ".join(stemmed_tokens)

print("original text", text)
print("stemmed text", stem_text)

original text running runners run easily jumper jumps jumping
stemmed text run runner run easili jumper jump jump


### Lemmatization

In [ ]:
def lemmatize_text(doc):
    return [token.lemma_ for token in doc]

lemmatized_tokens = lemmatize_text(single_doc)
print(lemmatized_tokens[:10])

['status', ':', ' ', 'this', 'be', 'the', 'original', 'version', '(', 'as']
